In [1]:
import os

In [2]:
%pwd

'/home/hawobade/MLOps/Anomaly-Detection-System-in-Network-Traffic/research'

In [3]:
%cd ..

/home/hawobade/MLOps/Anomaly-Detection-System-in-Network-Traffic


In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    dataset_ID: str


In [6]:
from src.anomalyDetection.constants import *
from src.anomalyDetection.utils.common import read_yaml, create_directories

In [49]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            dataset_ID=config.dataset_ID,
        )

        return data_ingestion_config

In [50]:
import os
import opendatasets as od
import zipfile  
from src.anomalyDetection import logger
from src.anomalyDetection.utils.common import get_size
from kaggle.api.kaggle_api_extended import KaggleApi
from dotenv import load_dotenv

In [70]:
load_dotenv()

os.environ['KAGGLE_USERNAME'] = os.getenv('MY_KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = os.getenv('MY_KAGGLE_KEY')

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_files(self):
        if not os.path.exists(self.config.local_data_file):
            api = KaggleApi()
            api.authenticate()
            api.dataset_download_files(
                self.config.dataset_ID,
                path=self.config.local_data_file,
                unzip=True
            )
            logger.info(f"{self.config.dataset_ID.split('/')[-1]} downloaded to {self.config.local_data_file}")
            logger.info(f"File size: {get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts rge zip file into the directory
        Function returns None
        """

        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)


In [71]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_files()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-20 03:54:25,805: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-20 03:54:25,808: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-20 03:54:25,810: INFO: common: created directory at: artifacts]
[2023-10-20 03:54:25,812: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-20 03:54:36,940: INFO: 236184812: cicids2017 downloaded to artifacts/data_ingestion/cicdataset]
[2023-10-20 03:54:36,942: INFO: 236184812: File size: ~ 4 KB]
